In [48]:
import pandas as pd
import plotly.express as px

In [49]:
df = pd.read_pickle("results.pkl")
df["node_pair"] = df["node_pair"].map(tuple).astype(str)
df["top_k"] = df["top_k"].astype(str)
df["Average Add Time"] = df["Add Times"].map(lambda x: sum(x) / len(x))
df["Average Perturb Time"] = df["Perturb Times"].map(lambda x: sum(x) / len(x))
# df["Path Selector"] = df["path_selector_class"]

In [50]:
df.to_csv("results.csv")

In [51]:
df.columns

Index(['Trial Number', 'Original Path Length', 'Time Taken', 'Perturbations',
       'Total Perturbation', 'Success', 'perturbation_function',
       'global_budget', 'local_budget', 'epsilon', 'k', 'node_pair', 'top_k',
       'max_iterations', 'path_selector_class', 'source', 'target', 'goal',
       'Path Selector', 'Number of Paths', 'Number of Edges', 'Add Times',
       'Perturb Times', 'Iterations', 'Final Distance', 'Status',
       'Average Add Time', 'Average Perturb Time'],
      dtype='object')

In [52]:
px.bar(df, x="node_pair", y="Time Taken", barmode="group", color="Path Selector", facet_col="top_k", facet_row="k", pattern_shape="Status", hover_data=["Average Add Time", "Average Perturb Time", "Original Path Length", "Number of Paths", "Number of Edges"])